Select the gene of interest, the species you want to compare, and the position of the residue.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import appmode
import pandas
from ipywidgets import FloatSlider
import os
from ipywidgets import Button, Layout

In [2]:
list_genes=pandas.read_csv("Available_genes.tab", sep="\t", header=None)
list_genes.columns=['Gene', 'File']
list_genes=list_genes.set_index('Gene').to_dict('index')


## Single Residue

In [16]:

directory = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='50%', height='30px', align_content='center'))
colonnes = widgets.Dropdown(disabled=False,layout=Layout(width='50%', height='30px', align_content='center'))
position = widgets.IntSlider(step=1,disabled=False,layout=Layout(width='80%', height='30px',align_content='center'))

# Updates the image options based on directory value
def update_columns(*args):
    table=pandas.read_csv(list_genes[directory.value]['File'], sep="\t")
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes.options = tmp

def update_pos(*args):
    table=pandas.read_csv(list_genes[directory.value]['File'], sep="\t")
    position.max=int(table[colonnes.value].max())
    
            
directory.observe(update_columns, 'value')
colonnes.observe(update_pos, 'value')



def show_position(Gene, Species,Position):
    if Gene!="":
        table=pandas.read_csv(list_genes[Gene]['File'], sep="\t")
        if len(table[table[Species]==Position])==0:
            print('No results')
        else:
            return table[table[Species]==Position]

_ = interact(show_position, Gene=directory, Species=colonnes, Position=position)

interactive(children=(Dropdown(description='Gene', layout=Layout(align_content='center', height='30px', width=…


## Region

In [21]:

directory_reg = widgets.Dropdown(options=[""]+list(list_genes.keys()),layout=Layout(width='50%', height='30px', align_content='center'))
colonnes_reg = widgets.Dropdown(disabled=False,layout=Layout(width='50%', height='30px', align_content='center'))
start = widgets.IntSlider(step=1,disabled=False,layout=Layout(width='80%', height='30px',align_content='center'))
end = widgets.IntSlider(step=1,disabled=False,layout=Layout(width='80%', height='30px',align_content='center'))

# Updates the image options based on directory value
def update_columns_reg(*args):
    table=pandas.read_csv(list_genes[directory_reg.value]['File'], sep="\t")
    tmp=list(table.columns)
    tmp.remove('AA')
    colonnes_reg.options = tmp

def update_start(*args):
    table=pandas.read_csv(list_genes[directory_reg.value]['File'], sep="\t")
    start.max=int(table[colonnes_reg.value].max())
    start.value=0

def update_max_start(*args):
    start.max=end.value

def update_min_end(*args):
    end.min=start.value


def update_end(*args):
    table=pandas.read_csv(list_genes[directory_reg.value]['File'], sep="\t")
    end.max=int(table[colonnes_reg.value].max())
    end.value=int(table[colonnes_reg.value].max())

directory_reg.observe(update_columns_reg, 'value')
colonnes_reg.observe(update_start, 'value')
colonnes_reg.observe(update_end, 'value')
end.observe(update_max_start, 'value')
start.observe(update_min_end, 'value')



def show_position_reg(Gene, Species,Start,End):
    if Gene!="":
        table=pandas.read_csv(list_genes[Gene]['File'], sep="\t")
        if len(table[(table[Species]>=Start) & (table[Species]<=End)])==0:
            print('No results')
        else:
            return table[(table[Species]>=Start) & (table[Species]<=End)]

_ = interact(show_position_reg, Gene=directory_reg, Species=colonnes_reg, Start=start,End=end )



interactive(children=(Dropdown(description='Gene', layout=Layout(align_content='center', height='30px', width=…